<a href="https://colab.research.google.com/github/SLVmain/PyTorch_start/blob/Les_03/GB_HW_03_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn.functional as F
from torch import nn
from torch import optim

import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm_notebook
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data  train.csv


In [ ]:
data = pd.read_csv('train.csv')
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [ ]:
data.columns

Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34808 entries, 0 to 34807
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   item_id               34808 non-null  object 
 1   user_id               34808 non-null  object 
 2   region                34808 non-null  object 
 3   city                  34808 non-null  object 
 4   parent_category_name  34808 non-null  object 
 5   category_name         34808 non-null  object 
 6   param_1               33447 non-null  object 
 7   param_2               19575 non-null  object 
 8   param_3               14705 non-null  object 
 9   title                 34808 non-null  object 
 10  description           32134 non-null  object 
 11  price                 32779 non-null  float64
 12  item_seq_number       34808 non-null  int64  
 13  activation_date       34808 non-null  object 
 14  user_type             34808 non-null  object 
 15  image              

In [ ]:
numeric_feature_names = ['price', 'item_seq_number', 'image_top_1', 'deal_probability']
numeric_features = data[numeric_feature_names]
numeric_features.head()

,price,item_seq_number,image_top_1,deal_probability
0,400.0,2,1008.0,0.12789
1,3000.0,19,692.0,0.00000
2,4000.0,9,3032.0,0.43177
3,2200.0,286,796.0,0.80323
4,40000.0,3,2264.0,0.20797


In [ ]:
for col in numeric_features.columns:
  if numeric_features[col].isna().sum() > 0:
    numeric_features[col].fillna(numeric_features[col].median(), inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
numeric_features.isna().sum()

price               0
item_seq_number     0
image_top_1         0
deal_probability    0
dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

numeric_features[['price']] = scaler.fit_transform(numeric_features[['price']])

numeric_features.head()

,price,item_seq_number,image_top_1,deal_probability
0,9.997493e-07,2,1008.0,0.12789
1,7.498120e-06,19,692.0,0.00000
2,9.997493e-06,9,3032.0,0.43177
3,5.498621e-06,286,796.0,0.80323
4,9.997493e-05,3,2264.0,0.20797


In [ ]:
numeric_features[['item_seq_number']] = scaler.fit_transform(numeric_features[['item_seq_number']])

In [ ]:
numeric_features[['image_top_1']] = scaler.fit_transform(numeric_features[['image_top_1']])

In [ ]:
numeric_features.head()

,price,item_seq_number,image_top_1,deal_probability
0,9.997493e-07,0.000005,0.328767,0.12789
1,7.498120e-06,0.000088,0.225701,0.00000
2,9.997493e-06,0.000039,0.988911,0.43177
3,5.498621e-06,0.001397,0.259622,0.80323
4,9.997493e-05,0.000010,0.738421,0.20797


In [ ]:
numeric_features.iloc[:, :-1]

,price,item_seq_number,image_top_1
0,9.997493e-07,0.000005,0.328767
1,7.498120e-06,0.000088,0.225701
2,9.997493e-06,0.000039,0.988911
3,5.498621e-06,0.001397,0.259622
4,9.997493e-05,0.000010,0.738421
...,...,...,...
34803,9.997493e-07,0.001348,0.405088
34804,4.248935e-05,0.000000,0.948793
34805,7.498120e-05,0.000029,0.951729
34806,9.495119e-06,0.000015,0.951729


In [ ]:
numeric_features.iloc[:, -1:]

,deal_probability
0,0.12789
1,0.00000
2,0.43177
3,0.80323
4,0.20797
...,...
34803,0.00000
34804,0.31978
34805,0.16079
34806,0.76786


In [ ]:
train, test = train_test_split(numeric_features, test_size=0.25, random_state=13)

надеюсь, будет разбор домашек, расскажете, как правильно сделать, потому что, скорее всего, обработку надо делать внутри класса датасет?

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df.copy()

        self.X = torch.Tensor(self.df.iloc[:, :-1].values)
        self.y = np.array(self.df.iloc[:, -1:].values.tolist())
    
    def __len__(self):
        return self.df.shape[0]


    def __getitem__(self, index):
        return (self.X[index], self.y[index])


In [ ]:
x_train = MyDataset(train)
x_test = MyDataset(test)

In [ ]:
train_loader = torch.utils.data.DataLoader(x_train, batch_size=16, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(x_test, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
for batch, label in train_loader:
    print(batch.shape)
    print(label.shape)
    print(label)
    break

torch.Size([16, 3])
torch.Size([16, 1])
tensor([[0.0000],
        [0.0000],
        [0.3609],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0955],
        [0.0000],
        [0.0684],
        [0.0000],
        [0.8652],
        [0.0000],
        [0.0000],
        [0.3551],
        [0.0000],
        [0.4000]], dtype=torch.float64)


In [ ]:
#F.tanh(x), F.leaky_relu(x, 0.05), F.relu(x)


class SigmoidNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.batch1 = nn.BatchNorm1d(4 * hidden_dim)
        self.dp = nn.Dropout(0.20)
        self.layer2 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        self.batch2 = nn.BatchNorm1d(2 * hidden_dim)
        self.layer3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.layer4 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        
        x = self.layer1(x)
        x = F.leaky_relu(x, 0.05)
        x = self.batch1(x)
        x = self.dp(x)
        x = self.layer2(x)
        x = F.leaky_relu(x, 0.05)
        x = self.batch2(x)
        x = self.dp(x)
        x = self.layer3(x)
        x = F.leaky_relu(x, 0.05)
        
        x = self.layer4(x)
        x = F.sigmoid(x)
        return x




In [ ]:
def rmseloss(y_pred, y_true, eps=1e-6):
  loss = torch.sqrt(torch.mean((torch.log(y_pred+eps)-torch.log(y_true+eps))**2))
  return loss

In [ ]:
net = SigmoidNN(3, 100)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm import tqdm

In [ ]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = rmseloss(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     1] loss: 0.005
[1,   301] loss: 0.983
[1,   601] loss: 0.874
[1,   901] loss: 0.867
[1,  1201] loss: 0.872
[1,  1501] loss: 0.861


 10%|█         | 1/10 [00:04<00:40,  4.50s/it]

[2,     1] loss: 0.002
[2,   301] loss: 0.862
[2,   601] loss: 0.870
[2,   901] loss: 0.869
[2,  1201] loss: 0.854
[2,  1501] loss: 0.862


 20%|██        | 2/10 [00:09<00:36,  4.53s/it]

...

[9,     1] loss: 0.003
[9,   301] loss: 0.849
[9,   601] loss: 0.856
[9,   901] loss: 0.854
[9,  1201] loss: 0.855
[9,  1501] loss: 0.855


 90%|█████████ | 9/10 [00:41<00:04,  4.55s/it]

[10,     1] loss: 0.003
[10,   301] loss: 0.844
[10,   601] loss: 0.848
[10,   901] loss: 0.862
[10,  1201] loss: 0.861
[10,  1501] loss: 0.847


100%|██████████| 10/10 [00:45<00:00,  4.56s/it]

Training is finished!


In [ ]:
net2 = SigmoidNN(3, 100) #чтобы сравнить оптимизаторы, инициализируем сеть заново
optimizer = torch.optim.Adam(net2.parameters(), lr=0.0004)

In [ ]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net2(inputs)
        loss = rmseloss(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     1] loss: 0.006
[1,   301] loss: 0.970
[1,   601] loss: 0.875
[1,   901] loss: 0.851
[1,  1201] loss: 0.869
[1,  1501] loss: 0.865


 10%|█         | 1/10 [00:06<00:56,  6.24s/it]

[2,     1] loss: 0.003
[2,   301] loss: 0.860
[2,   601] loss: 0.859
[2,   901] loss: 0.855
[2,  1201] loss: 0.860
[2,  1501] loss: 0.853


 20%|██        | 2/10 [00:12<00:49,  6.19s/it]

...

[9,     1] loss: 0.002
[9,   301] loss: 0.840
[9,   601] loss: 0.849
[9,   901] loss: 0.852
[9,  1201] loss: 0.841
[9,  1501] loss: 0.857


 90%|█████████ | 9/10 [00:58<00:06,  6.48s/it]

[10,     1] loss: 0.003
[10,   301] loss: 0.853
[10,   601] loss: 0.850
[10,   901] loss: 0.831
[10,  1201] loss: 0.856
[10,  1501] loss: 0.842


100%|██████████| 10/10 [01:04<00:00,  6.47s/it]

Training is finished!


In [ ]:
net3 = SigmoidNN(3, 100) #чтобы сравнить оптимизаторы, инициализируем сеть заново
optimizer = torch.optim.RMSprop(net3.parameters())

for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net3(inputs)
        loss = rmseloss(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,     1] loss: 0.006
[1,   301] loss: 1.098
[1,   601] loss: 1.104
[1,   901] loss: 1.119
[1,  1201] loss: 1.108
[1,  1501] loss: 1.097


 10%|█         | 1/10 [00:05<00:50,  5.57s/it]

[2,     1] loss: 0.004
[2,   301] loss: 1.109
[2,   601] loss: 1.094
[2,   901] loss: 1.161
[2,  1201] loss: 1.105
[2,  1501] loss: 1.105


 20%|██        | 2/10 [00:11<00:47,  5.95s/it]

...

[9,     1] loss: 0.004
[9,   301] loss: 1.103
[9,   601] loss: 1.107
[9,   901] loss: 1.109
[9,  1201] loss: 1.095
[9,  1501] loss: 1.098


 90%|█████████ | 9/10 [01:25<00:12, 12.11s/it]

[10,     1] loss: 0.004
[10,   301] loss: 1.127
[10,   601] loss: 1.113
[10,   901] loss: 1.109
[10,  1201] loss: 1.100
[10,  1501] loss: 1.082


100%|██████████| 10/10 [01:41<00:00, 10.11s/it]

Training is finished!


# На 10 эпохах немного лучше сходимость у ADAM, затем SDG, хуже всего - у RMSprop

In [ ]:
"""class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        return loss"""

In [ ]:
#train = pd.read_csv('../train.csv.zip',compression='zip',parse_dates=['activation_date'])
#test = pd.read_csv('../test.csv.zip',compression='zip',parse_dates=['activation_date'])

#x = x.view(x.shape[0], -1)